In [1]:
import graphviz

class DFA:
    def __init__(self, num_states, alphabet, transitions, initial_state, final_states):
        self.num_states = num_states
        self.alphabet = alphabet
        self.transitions = transitions
        self.initial_state = initial_state
        self.final_states = final_states

    def minimize(self):
        reachable = self.get_reachable_states()
        non_final_states = sorted(reachable - set(self.final_states))
        final_states = sorted(reachable & set(self.final_states))
        partitions = [non_final_states, final_states]
        state_in_partition = {state: index+1 for index, partition in enumerate(partitions) for state in partition}
        while True:
            new_partitions = []
            new_in_partition = {}
            for partition in partitions:
                if len(partition) == 1:
                    new_partitions.append(partition)
                    new_in_partition[partition[0]] = len(new_partitions)
                    continue
                group = {}
                for state in partition:
                    signature = tuple(state_in_partition[self.transitions[state][symbol]] for symbol in self.alphabet)
                    group.setdefault(signature, []).append(state)
                new_partitions.extend(group.values())
                for index, grp in enumerate(group.values(), start=len(new_partitions) - len(group.values()) + 1):
                    for state in grp:
                        new_in_partition[state] = index
            if len(new_partitions) == len(partitions):
                break
            partitions = new_partitions
            state_in_partition = new_in_partition
        return partitions, state_in_partition

    def get_reachable_states(self):
        reachable = {self.initial_state}
        stack = [self.initial_state]
        while stack:
            current = stack.pop()
            for symbol in self.alphabet:
                next_state = self.transitions[current][symbol]
                if next_state and next_state not in reachable:
                    reachable.add(next_state)
                    stack.append(next_state)
        return reachable

    def print_transition_table(self, partitions, state_in_partition):
        print("\nTRANSITION TABLE\n")
        header = " Symbol  ->  " + "      ".join(self.alphabet)
        print(header)
        for i, part in enumerate(partitions, start=1):
            print(f" Z{i}        ", end="")
            for symbol in self.alphabet:
                state = self.transitions[part[0]][symbol]
                print(f" Z{state_in_partition[state]}     " if state_in_partition[state] else "      -      ", end="")
            print()

    def draw_state_diagram(self, partitions, state_in_partition):
        d = graphviz.Digraph()
        d.node("start", label="", shape="none")
        d.edge("start", f"P{state_in_partition[self.initial_state]}", arrowhead="normal")
        for i, partition in enumerate(partitions, start=1):
            label = ' '.join(map(str, partition))
            shape = 'doublecircle' if any(state in partition for state in self.final_states) else 'circle'
            d.node(f"P{i}", label=label, shape=shape)
            for symbol in self.alphabet:
                next_partition = state_in_partition.get(self.transitions[partition[0]][symbol], None)
                if next_partition:
                    d.edge(f"P{i}", f"P{next_partition}", label=symbol)
        d.render("state_diagram", format="png", cleanup=True)

def get_input(prompt, data_type=int):
    while True:
        try:
            user_input = data_type(input(prompt))
            return user_input
        except ValueError:
            print("Invalid input. Please enter again.")

def minimize_dfa():
    num_states = get_input("Enter the total number of states: ")
    alphabet = input("Enter the valid input symbols separated by space: ").strip().split()
    transitions = {i+1: {symbol: get_input(f"For state {i+1}, transition on entering {symbol}: ") for symbol in alphabet} for i in range(num_states)}
    initial_state = get_input("Enter the index of the initial state: ")
    num_final_states = get_input("Enter the total number of final states: ")
    final_states = [get_input("Enter the index of a final state: ") for _ in range(num_final_states)]

    dfa = DFA(num_states, alphabet, transitions, initial_state, final_states)
    partitions, state_in_partition = dfa.minimize()

    print("Number of equivalence states partitions:", len(partitions))
    print("Partitions are:")
    for index, partition in enumerate(partitions, start=1):
        print(f"Z{index}: {' '.join(map(str, partition))}")

    dfa.print_transition_table(partitions, state_in_partition)
    dfa.draw_state_diagram(partitions, state_in_partition)

minimize_dfa()


Enter the total number of states: 6
Enter the valid input symbols separated by space: 1 0
For state 1, transition on entering 1: 2
For state 1, transition on entering 0: 4
For state 2, transition on entering 1: 6
For state 2, transition on entering 0: 3
For state 3, transition on entering 1: 6
For state 3, transition on entering 0: 3
For state 4, transition on entering 1: 5
For state 4, transition on entering 0: 1
For state 5, transition on entering 1: 6
For state 5, transition on entering 0: 3
For state 6, transition on entering 1: 6
For state 6, transition on entering 0: 6
Enter the index of the initial state: 1
Enter the total number of final states: 3
Enter the index of a final state: 2
Enter the index of a final state: 3
Enter the index of a final state: 5
Number of equivalence states partitions: 3
Partitions are:
Z1: 1 4
Z2: 6
Z3: 2 3 5

TRANSITION TABLE

 Symbol  ->  1      0
 Z1         Z3      Z1     
 Z2         Z2      Z2     
 Z3         Z2      Z3     
